In [9]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from pathlib import Path

df = pd.read_csv(os.path.join(*'../../datasets/zalando_outfit_clothes/woman/woman_outfit_clothes.csv'.split('/')))
df

,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl
0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...
1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...
2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...
3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...
4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...
...,...,...,...,...,...,...
24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...
24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...
24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...
24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...


In [10]:
(df.shape[0] * 5) / 60 / 60

34.09861111111111

In [11]:
len(list(set(df['outfit_url'])))

6746

## Extract outfit ids from url

In [12]:
import re


def extract_id_from_url(url):
    # Regular expression pattern to extract the ID from the URL
    pattern = r'/outfits/([\w-]+)/?$'

    # Using regex to find the ID
    match = re.search(pattern, url)

    # Check if a match is found
    if match:
        return match.group(1)  # Extract the matched ID
    else:
        return None  # Return None if no match is found


# Example URL
url = "https://www.zalando.ch/outfits/5Bay-ErCSwa/"

# Extracting ID from the URL
outfit_id = extract_id_from_url(url)

# Displaying the extracted ID
print("Extracted ID:", outfit_id)


Extracted ID: 5Bay-ErCSwa


## Remove query parameters from image url to get full quality image

In [13]:
df['id'] = df['outfit_url'].apply(extract_id_from_url)
df['clothingItemPreviewImageUrl'] = df['clothingItemPreviewImageUrl'].apply(lambda x: x.split('?')[0])
df

,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id
0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa
1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa
2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa
3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa
4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa
...,...,...,...,...,...,...,...
24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu
24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu
24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm
24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6


## Define Methods to save images and get color of clothing item from webpage

In [14]:

def save_clothing_item_image_from_dataframe_record(data_frame_record, item_index):
    outfit_dir = f'woman_outfits/{data_frame_record.id}'

    Path(f'{outfit_dir}').mkdir(parents=True, exist_ok=True)

    img_data = requests.get(
        data_frame_record.clothingItemPreviewImageUrl
    ).content

    image_path = os.path.join(*f'{outfit_dir}/{item_index}.jpg'.split('/'))

    with open(image_path, 'wb') as handler:
        handler.write(img_data)

    return image_path


def get_color_of_clothing_item_from_detail_page_url(url):
    clothing_item_detail_page = requests.get(
        url
    )
    soup = BeautifulSoup(clothing_item_detail_page.content, "html.parser")
    colorLabel = [element for element in soup.select('.sDq_FX.lystZ1.FxZV-M.HlZ_Tf') if element.get_text() == 'Farbe:'][
        0]
    return colorLabel.parent.parent.select_one('.sDq_FX.lystZ1.dgII7d.HlZ_Tf.zN9KaA').get_text()

# Start Crawling

In [15]:
import numpy as np

df["clothingItemColor"] = np.nan
df["clothingItemColor"] = df["clothingItemColor"].astype(str)
df["image_path"] = np.nan
df["image_path"] = df["image_path"].astype(str)
df

,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path
0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,nan,nan
1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,nan,nan
2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,nan,nan
3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,nan,nan
4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,nan,nan
...,...,...,...,...,...,...,...,...,...
24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,nan,nan
24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,nan,nan
24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,nan,nan
24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,nan,nan


In [16]:
from time import sleep
from tqdm import tqdm
import random

number_of_clothes = df.shape[0]

clothing_items_for_df = []

print("Start crawling clothes info and images")

outfit_ids = list(set(df['id']))

for i in tqdm(range(len(outfit_ids))):
#for i in tqdm(range(3)):
    outfit_id = outfit_ids[i]
    clothes = df[df['id'] == outfit_id]
    item_index = 0

    for index, row in clothes.iterrows():
        try:
            image_path = save_clothing_item_image_from_dataframe_record(row, item_index)
            df.loc[index, 'image_path'] = image_path
        except Exception as err:
            print(f"Error saving image: for outfit {outfit_id} item: {item_index}")

        sleep(random.uniform(0.5, 1))
        
        try:
            df.loc[index, 'clothingItemColor'] = get_color_of_clothing_item_from_detail_page_url(
            row['clothingItemDetailUrl'])
        except Exception as err:
            print(f"Error getting color: for outfit {outfit_id} item: {item_index}")

        sleep(random.uniform(3, 8))
        item_index += 1

df.to_csv('woman_outfit_clothes_with_img_file_path_and_color.csv')
print("Finished crawling, wrote csv file")

Start crawling clothes info and images


  0%|          | 13/6746 [04:27<38:39:54, 20.67s/it]

Error getting color: for outfit pgR9mSrVRY6 item: 0


  0%|          | 27/6746 [10:49<55:06:54, 29.53s/it]

Error getting color: for outfit tcv_XKojRza item: 0


...

Error getting color: for outfit 2caAAAi-QDK item: 1


 99%|█████████▊| 6658/6746 [49:38:41<21:28, 14.64s/it]  

Error getting color: for outfit n-2OR4nJQzK item: 0


100%|██████████| 6746/6746 [50:15:48<00:00, 26.82s/it]


Finished crawling, wrote csv file


In [82]:
df[df['id'] == 'YKUbP7mNRky']

,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path
15697,15697,https://www.zalando.ch/outfits/YKUbP7mNRky/,GRAPHIC RICKIE TEE - T-Shirt basic - lilac/red,Levi's®,https://www.zalando.ch/levis-graphic-rickie-te...,https://img01.ztat.net/article/spp-media-p1/18...,YKUbP7mNRky,lilac/red,woman_outfits/YKUbP7mNRky/0.jpg
15698,15698,https://www.zalando.ch/outfits/YKUbP7mNRky/,LONG - Maxirock - dark blue,PULL&BEAR,https://www.zalando.ch/pullandbear-jeansrock-d...,https://img01.ztat.net/article/spp-media-p1/21...,YKUbP7mNRky,dark blue,woman_outfits/YKUbP7mNRky/1.jpg
15699,15699,https://www.zalando.ch/outfits/YKUbP7mNRky/,CLUB C GROUNDS UNISEX - Sneaker low - navy/chalk,Reebok Classic,https://www.zalando.ch/reebok-classic-club-gro...,https://img01.ztat.net/article/spp-media-p1/06...,YKUbP7mNRky,navy/chalk,woman_outfits/YKUbP7mNRky/2.jpg
15700,15700,https://www.zalando.ch/outfits/YKUbP7mNRky/,KOTTI UNISEX - Sonnenbrille - black/gradient,CHPO,https://www.zalando.ch/chpo-kotti-unisex-sonne...,https://img01.ztat.net/article/spp-media-p1/a2...,YKUbP7mNRky,black/gradient,woman_outfits/YKUbP7mNRky/3.jpg
15701,15701,https://www.zalando.ch/outfits/YKUbP7mNRky/,Handtasche - gold,Even&Odd,https://www.zalando.ch/evenandodd-handtasche-g...,https://img01.ztat.net/article/spp-media-p1/f9...,YKUbP7mNRky,gold,woman_outfits/YKUbP7mNRky/4.jpg
